# SI 618: Data Manipulation and Analysis
## 04 - Univariate Statistics & Visualization

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a> This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.


# Overview of today 

- Announcements -> Upcoming Homework
- Comments/Questions/Concerns
- Review last week: Aggregation & Grouping
- Today: Univariate Statistics  & Vizualization

### IMPORTANT: Replace ```?``` in the following code with your uniqname.

In [ ]:
MY_UNIQNAME = 'tengdann'

# ----------Aggregation and Grouping----------

## Learning Objectives
* use StringIO to create a DataFrame
* use the .describe() function
* understand .groupby()
* understand different types of merges

## Build the dataframe

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
names = ['Gandalf',
         'Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Aragorn',
         'Galadriel',
         'Meriadoc',
        'Lily']
races = ['Maia',
         'Dwarf',
         'Hobbit',
         'Elf',
         'Hobbit',
         'Hobbit',
         'Hobbit',
         'Man',
         'Man',
         'Elf',
         'Hobbit',
        'Hobbit']
magic = [10, 1, 4, 6, 4, 2, 0, 0, 2, 9, 0, np.NaN]
aggression = [7, 10, 2, 5, 1, 6, 3, 8, 7, 2, 4, np.NaN ]
stealth = [8, 2, 5, 10, 5, 4 ,5, 3, 9, 10, 6, np.NaN]

In [ ]:
df = pd.DataFrame({'name': names,'race':races,'magic':magic,'aggression': aggression,'stealth':stealth})

## Joining Data



Let's say we have another CSV file that contains URLs to Wikipedia pages for some of the LOTR characters:

In [ ]:
urls = pd.read_csv('data/lotr_wikipedia.csv')

Let's take a look at the original DataFrame:

It looks like the rows are "aligned", so we can use the ```concat``` function to concatenate the two DataFrames.
Note that we specify the axis to be the columns.  The default is to concatenate by rows, which isn't what we want.

In [ ]:
pd.concat([df,urls],axis="columns")

That's great, and it's consistent with what we've used in previous classes.  But what happens if the 
rows in the two DataFrames don't match up?  Let's load another file that has a slightly different
sequence of rows:

In [ ]:
urls_wrong_order = pd.read_csv('data/lotr_wikipedia_wrong_order.csv')

In [ ]:
urls_wrong_order

In [ ]:
pd.concat([df,urls_wrong_order],axis="columns")

Take a closer look at the name and url columns.  Something's not quite right.

We can work around that by using the appropriate indexing and then using the SQL-like ```join``` function. ```.join``` Join columns with other DataFrame either on index or on a key column. By default, ```join``` uses a **left** join, which means the all the values from the "left" side are used, whether or not there's a corresponding entry from the "right" side. 

In [ ]:
df_names = df.set_index('name')

In [ ]:
urls_wrong_order_names = urls_wrong_order.set_index('name')

In [ ]:
df_names.join(urls_wrong_order_names)

In [ ]:
urls_wrong_order.head()

In [ ]:
urls_wrong_order['name']

In [ ]:
df['name']

In [ ]:
df.merge(urls_wrong_order,on='name')

## Let's add some more data

Now let's add a few additional URLs:

In [ ]:
urls_extras = pd.read_csv("data/lotr_wikipedia_extras.csv")

And now let's use concat to add the new entries to the DataFrame.

In [ ]:
urls_complete = pd.concat([urls,urls_extras])

In [ ]:
urls_complete

## Merging Data

![pivot 1](http://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png "pivots")

By default, ```merge``` uses an **"inner"** join, which include only those values that exist in both the "left" and "right" DataFrames:

In [ ]:
df.merge(urls_complete,on='name',how='inner')

A left join, which means the all the values from the **"left"** side are used, whether or not there's a corresponding entry from the "right" side.  In the example below, note that the url value for "Lily" is "NaN":

In [ ]:
df.merge(urls_complete,on='name',how='left')

The "opposite" of a left join is, perhaps unsurprisingly, a **"right"** join, in which
all the values from the "right" side are used, whether or not a corresponding
value from the "left" side exists. Note in the following example that "Lily" has
disappeared, and Treebeard and Elrond lack information about "race", "magic", "aggression", and "stealth".

In [ ]:
df.merge(urls_complete,on='name',how='right')

In addition to "left" and "right" joins, we have **"outer"** joins, which include
values from both the "left" and "right" DataFrames, regardless of whether
there are corresponding values in the other DataFrame.  Note that all of 
"Lily", "Treebeard" and "Elrond" are present in the following DataFrame:

In [ ]:
df.merge(urls_complete,on='name',how='outer')

Sometimes it's nice to know how a particular row got added to the resulting DataFrame.  Using **```indicator=True```**
allows us to examine this:

In [ ]:
df.merge(urls_complete,how='outer',indicator=True)

You'll note that we used the ```merge``` function from the DataFrame and passed in the other DataFrame as an argument.
You can also call the ```merge``` function from pandas directly and pass it the two DataFrames you are merging:

In [ ]:
pd.merge(df,urls_complete,how='outer',indicator=True)

## Group By

In [ ]:
# to start let's make a fake dataset: sales of fruit across US states.
# Don't worry about the details here, but basically we'll pretend
# this string is a CSV file and use the standard loading ops
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales
MI,Walmart,Apple,100
MI,Wholefoods,Apple,150
MI,Kroger,Orange,180
CA,Walmart,Apple,220
CA,Wholefoods,Apple,180
CA,Safeway,Apple,220
CA,Safeway,Orange,110
NY,Walmart,Apple,90
NY,Walmart,Orange,80
NY,Wholefoods,Orange,120
""")

fruit = pd.read_csv(TESTDATA, index_col=None)

In [ ]:
fruit.head()

## (a) What is the total sales for each state?
This requires us to group by state, and aggregate sales by taking the sum.

The easiest way of doing this if to use `groupby`

If you execute groupby on the dataframe what you'll get back is an object called DataFrameGroupBy

In [ ]:
fruit.groupby('State')

On its own it's a bit useless... it just keeps track of which rows should go into each "pile" (where pile here means a unique group for each state)

If we ask this object to describe itself, you can see what is inside notice that it threw away all the other columns because they were not numerical.  Only "Sales" which is a number, was kept

In [ ]:
fruit.groupby('State').describe()

In [ ]:
# What are the total sales for each state?
fruit.groupby('State').sum()  # instead of size()

What just happend? A couple of things:
- `groupby()` got first executed on `df`, returning an `DataFrameGroupBy` object. This object itself is useless unless coupled with an aggregation function, such as `sum()`, `mean()`, `max()`, `apply()`. We will talk about `apply()` more in the next week.
- Then, `sum()` got executed on the `DataFrameGroupBy` object, generating the `DataFrame` object you see above. Notice how the table looks different than the original DataFrame `df`? Here are the differences:
  - The `State` column now becomes the index of the DataFrame. The string "State" is the name of the index. Notice how the index name is displayed on a lower level than column names.
  - Since we performed a `groupby` operation by `State`, so only the unique values of `State` are kept as index.
  - Among the other columns, Retailer, Fruit, and Sales, only Sales is kept in the result table. This is because the aggregation function `sum()` only knows how to aggregate numerical values. And only Sales is a numerical column. The other columns are hence dropped.

## (b) What is the total sales for each state for each fruit?
This requires us to perform `groupby` on two columns. So, we provide a list of column names to the `groupby` function.

Don't forget that an aggregation function needs to follow the `groupby` function in order to generate results.

In [ ]:
# What is the total sales for each state for each fruit?
fruit.groupby(['State','Fruit']).sum()

How is this DataFrame different from the previous one?

The biggest different is that this DataFrame has what is called a `MultiIndex` (or hierarchical index), as opposed to a simple index. In this table, the left two "columns" are not columns but actually part of the `MultiIndex`, and the `Sales` is the single real "column" in the DataFrame. (Running out of terminologies here...)

The hierarchical index can be organized in an alternative way if we swapped the order of State and Fruit.

In [ ]:
fruit.groupby(['Fruit','State']).sum()

## (c) Which state has the maximum total sales?
This question is not asking about the maximum value, but rather which state holds that maximum. There are multiple ways to do it. A principled way is to use `idxmax`.

In [ ]:
# Which state has the maximum total sales?
fruitSalesByState = fruit.groupby('State').sum()
print(fruitSalesByState)
max_state = fruitSalesByState['Sales'].idxmax()
print("The state with the maximum sales is: ",max_state)

What if we wanted to get the sales value of CA again?

In [ ]:
fruitSalesByState['Sales'][max_state]

## (d) Which state has the maximum total sales for apples?</font>

In [ ]:
# Which state has the maximum total sales for apples?
# give me apple sellers
apples = fruit[fruit.Fruit == 'Apple']
apples

In [ ]:
# aggr. by state
applesByState = apples.groupby('State').sum()
applesByState

In [ ]:
applesByState.Sales

In [ ]:
applesByState.Sales.idxmax()

In [ ]:
applesByState.loc[applesByState.Sales.idxmax()]

In the above command, `.loc[]` looks up the index label and returns that row.



# ----------Univariate Statistics----------

In [ ]:
MY_UNIQNAME = 'schenry'

# A quick note about Markdown cells
We have encouraged you to use Markdown cells to add text to your notebooks.  Please see 
https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html for a more complete explanation of the use of Markdown in Jupyter.  You can also examine any of the existing Markdown blocks by clicking on them.

# So, you want to explore your data...
* How can we describe it?
* What does it look like?
* What sorts of "preliminary checks" can we perform on our data? 
 * Why would we want to to this?

# Eyeballing your data: plotting
* Introducing [Seaborn](http://seaborn.pydata.org/)
> Seaborn is a library for making attractive and informative statistical graphics in Python. It is built on top of matplotlib and tightly integrated with the PyData stack, including support for numpy and pandas data structures and statistical routines from scipy and statsmodels.

Um, ok.  How about [some examples](http://seaborn.pydata.org/examples/index.html)?

# Think about plotting the relationship between X and Y
![](resources/AnscombeData.png)
(https://en.wikipedia.org/wiki/Anscombe's_quartet)

# A nice linear relationship, right?
![](resources/AnscombeQ1.png)
(https://en.wikipedia.org/wiki/Anscombe's_quartet)

# Anscombe's Quartet
![](resources/AnscombePlot.png)
(https://en.wikipedia.org/wiki/Anscombe's_quartet)

# Why do we care?

* Statistical analysis requires good inputs
 * Remember Anscombe’s quartet
 * Which model should we use?
 * Tendencies/trends/patterns in data are important in picking the right models
* Anomalies are important to detect and understand




## Purpose of EDA (broadly)
* maximize insight into a data set
* uncover underlying structure
* extract important variables
* detect outliers and anomalies
* test underlying assumptions
* develop parsimonious models and
* determine optimal factor settings.

(National Institute of Standards and Technology)

## Our primary goal in this class: understand the _downstream_ statistical analyses enough so that we can make good decisions

# Spherical cows
The phrase comes from a joke that spoofs the simplifying assumptions that are sometimes used in theoretical physics.

>Milk production at a dairy farm was low, so the farmer wrote to the local university, asking for help from academia. A multidisciplinary team of professors was assembled, headed by a theoretical physicist, and two weeks of intensive on-site investigation took place. The scholars then returned to the university, notebooks crammed with data, where the task of writing the report was left to the team leader. Shortly thereafter the physicist returned to the farm, saying to the farmer, "I have the solution, but it works only in the case of spherical cows in a vacuum". [Wikipedia](https://en.wikipedia.org/wiki/Spherical_cow)

![Spherical cow](resources/Spot_the_cow.gif)




## The average weight of a Jersey cow is 1,000 lbs.
![](resources/jersey.jpg)

# Let's make some spherical cows...

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import seaborn as sns

In [ ]:
# The weights of a herd of 80 cows
measures = (np.random.standard_normal(80)*150+1000).astype(int)
measures

# <font color="red">BEGIN Q1: Create some objects of your own
Think of some object and some property of that object.  I used cows and their weights.  Pick something that you know something about, and create a NumPy array of some number of them (ideally between 20 and 50000), recording some property.  So you might choose something like the height of undergraduate students, etc.  Pick somethink that will *likely* have a normal distribution (which is probably most things you can think of.</font>

First, pick the number that you want and assign it to numberOfObjects, then pick the mean value and assign it to meanOfProperty, and finally pick the variance and assign it to varianceOfProperty.  It doesn't matter what you pick, but if you're unsure pick 1/5 of the mean.

In [ ]:
numberOfObjects = np.NaN    # change np.NaN to some number between 20 and 50000
meanOfProperty = np.NaN     # change np.NaN to the mean value of the property you're interested in
standardDeviationOfProperty = meanOfProperty/np.NaN  # change np.NaN to the variance of the property -- try meanOfProperty/5 if you have no idea

Now create some data by asking for a random sample from a normal distribution, scaled so that it matches the mean and variance you want:

In [ ]:
things = np.random.standard_normal(numberOfObjects)*standardDeviationOfProperty+meanOfProperty
# And let's say we want integers instead of floats:
things = things.astype(int)
things

_Explain your answer here._

# <font color="red">END Q1</font>

## Measures of central tendency
* Mean
* Median 
* Mode

## Mean

Add up all the values and divide by the number of values:

$$mean = \frac {\sum{x_i}} {n}$$


In [ ]:
sum(measures)/len(measures)

In [ ]:
np.mean(measures) # find the mean

## Median

sort all the numbers and find the one in the middle

In [ ]:
measures = np.sort(measures)
measures

In [ ]:
measures[len(measures)//2]  # find middle value

In [ ]:
np.median(measures)

## Another alternative: Mode

* mode = most common value
* Unfortunately not in default numpy


In [ ]:
from scipy import stats
stats.mode(measures)

# <font color="red">BEGIN Q2: Calculate the mean, median and mode of your "things"

Step 1: Just run the following cell (assumes you have some data in an np array called "things")

In [ ]:
from scipy import stats # just in case we didn't already do it

thingsMean = 0 # replace 0 with your code
thingsMedian = 0 # replace 0 with your code
thingsMode = stats.mode(0)[0].item() # replace 0 with your code

print(thingsMean, thingsMedian, thingsMode)

Step 2: Now, to demonstrate what happens to mean, median and mode when you add an outlier, append some crazy big value to the end of your things.  But let's not mess up our things array, so let's copy it first

In [ ]:
outliers = np.nan # change np.nan to some extreme value

things2 = things.copy()
things2 = np.append(things2,outliers)
things2Mean = np.mean(things2).round(2)
things2Median = np.median(things2)
things2Mode = stats.mode(things2)[0].item()

print(things2Mean, things2Median, things2Mode)

Step 3: Record, in your own words, what happened to each of the mean, median and mode when you added that value:

_Explain your answer here._

# <font color="red">END Q2</font>

## Measures of dispersion

* Percentile cutoffs
 * Interpercentile range
* Variance
* Standard Deviation

## Percentiles

* In a *sorted* list, find the threshold so that data is split
 * 5th percentile -- bottom 5% of measures below threshold
 * 25th percentile -- bottom 25% of measures below
 * 97th percentile -- bottom 97% of mesures below

## Numpy does this well

`np.percentile(array,percentile,
               interpolation='linear')`

* linear: i + (j - i) 
* fraction, where fraction is the fractional part of the index surrounded by i and j.
* lower: i.
* higher: j.
* nearest: i or j, whichever is nearest.
* midpoint: (i + j) / 2.


In [ ]:
np.percentile(measures,25) # the 25th percentile

In [ ]:
np.percentile(measures,25,interpolation='higher') # bump it up to the next higher real value from the data

## Interpercentile Range

* Sometimes we want to some range
 * e.g., 5th -- 95th percentile: 90% of measures sit here

In [ ]:
print(np.percentile(measures,5),"-",np.percentile(measures,95))

## Variance

How does the data spread around the mean?

$$ variance = \frac{\sum{(x_i - \mu)^2}}{n}$$

where, $\mu$ is the mean

$$ mu = \frac{\sum{x_i}}{n}$$

## Standard Deviation

* Measure of dispersion 

![standard deviation](resources/Standard_deviation_diagram.svg.png)
(https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Standard_deviation_diagram.svg/640px-Standard_deviation_diagram.svg.png "sd")

In [ ]:
print(np.percentile(measures,2.5),"-",np.percentile(measures,97.5))
print(np.var(measures))
print(np.std(measures))  # this should be the square root of variance

# <font color="red">BEGIN Q3: Measures of dispersion</font>
Examine the 2.5-97.5 percentile range and the standard deviation of your "things".
Does the output for standard deviation match what you asked for when you first generated the data in Q1? What's the relationship between the 2.5th-to-97.5th percentile range and the standard deviation? Answer below.

In [ ]:
#Enter your code here

_Explain your answer here._

# <font color="red">END Q3</font>

# Visualizing data with Seaborn
* Visualization package built on top of matplotlib
* It's meant to make your life better
* Plays well with pandas, numpy, scipy, and statsmodels
* Many different visualization are included:
 * Strip plots, Swarm plots, Violin plots
 * Box plots
 * Histograms
 
 We need to import the package, and it's typically imported as sns:
 
 ```
 import seaborn as sns
 ``` 
 
 and don't forget to inline matplotlib (that's a jupyter thing):

 ```
 %matplotlib inline
 ```

[seaborn.pydata.org](http://seaborn.pydata.org)

In [ ]:
import seaborn as sns

In [ ]:
print(sns.__version__)

In [ ]:
# If needed update to 0.8 !conda update -y seaborn 

## Strip Plot

In [ ]:
%matplotlib inline
import seaborn as sns  # you might need to do: conda install seaborn
sns.stripplot(x=measures)

## Swarm Plot

In [ ]:
sns.swarmplot(x=measures)

## Violin Plot
* If we have too much data

In [ ]:
sns.violinplot(x=measures,color="#d7d0f3")

## Box Plot

In [ ]:
sns.boxplot(x=measures) 

And we can manipulate the underlying plot to control different features.  See 
https://stackoverflow.com/questions/34162443/why-do-many-examples-use-fig-ax-plt-subplots-in-matplotlib-pyplot-python#34162641
and
https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.subplots for explanations about ```plt.subplots()```



In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
fig.set_size_inches(11, 8)
medianprops = dict( linewidth=2.5, color='firebrick')
sns.boxplot(medianprops=medianprops,data=measures,orient='h')
plt.xticks(rotation=90)


## Histogram

We're going to use this a lot.  Seaborn puts a nice smooth line over a distribution.  We'll talk about that soon, but for now just think about it as an extrapolation:  if we had a bunch more data, the distribution would eventually smooth out to something that looks like the line.

In [ ]:
# x axis = value, y axis = count (frequency)
sns.distplot(measures, kde=False); 

In [ ]:
sns.distplot(measures, rug=True); # show a strip plot on bottom -- we call it a "rug"

# <font color="red">BEGIN Q4: Test driving Seaborn</font>
Your turn:  create the above plots (strip, swarm, violin, box, and histogram for your "things".

In [ ]:
%matplotlib inline
import seaborn as sns  # you might need to do: conda install seaborn
import matplotlib.pyplot as plt # in case we want to manipulate the underlying matplotlib layer

In [ ]:
#Enter your code here

Chose two plots from the ones you generated above and in your own words explain what each of them tells you about your data.

_Explain your answer here._

# <font color="red">END  Q4</font>

## Expressiveness, Effectiveness, Scale

* Expressiveness
 * What facts can we extract?
 * What facts *can't* we extract?
* How well can we extract them?
* How do they work with more data?
 * Consider different dimensions
 * More samples
 * More univariate data

## Why do we care about distributions?
## World is not normal
* Many other kinds of distributions
* We can tell what they are by looking at distribution

## Uniform

In [ ]:
uniform = np.random.uniform(-2,2,1000)  # low,high,count
sns.distplot(uniform,kde=False,norm_hist=False)

## Bimodal

In [ ]:
bimodal = np.append(np.random.normal(-20,10,100),
                    np.random.normal(20,10,100))
sns.distplot(bimodal,kde=False,norm_hist=True)

# Poisson

$$ P(k~events~in~interval) = \frac{\lambda^ke^{-\lambda}}{k!} $$

$\lambda$ is the event rate

Examples
* Meteor strikes
* Arrival of patients to hospital

In [ ]:
# as lambda goes up --> looks more normal
pois = np.random.poisson(3,100000) # lambda, count 
sns.distplot(pois,kde=False,bins=10,norm_hist=True)

# Power/Zipf/Pareto

$$ P = \frac{x^{-a}}{\zeta(a)}$$

"long tail"
* degree distribution
* movie/music popularity
* words

## Note:  both axes are log-transformed

In [ ]:
# bit of a hack (seaborn)
power = np.random.zipf(2,100000)
ax = plt.plot(np.histogram(power,bins=400)[0])
ax[0].axes.set_xscale("log")
ax[0].axes.set_yscale("log")

## Visual Tests on Data

In [ ]:
testdata = (np.random.standard_normal(500)*20+150).astype(int)
sns.distplot(testdata)

## Run Sequence
* Run Sequence (index versus value)
* flat and non-drifting
 * fixed-location assumption holds
* vertical spread same over the entire plot, 
 * then the fixed-variation assumption holds.

In [ ]:
ax = sns.regplot(np.arange(len(testdata)),testdata,fit_reg=False)
ax.set_ylim(0,250)
ax.set_ylabel("val")
ax.set_xlabel("i")

In [ ]:
drifting = np.array([testdata[i]+i*(.1) for i in np.arange(len(testdata))])
sns.distplot(drifting,kde=False)

In [ ]:
ax = sns.regplot(np.arange(len(drifting)),drifting,fit_reg=False)
ax.set_ylim(0,300)
ax.set_ylabel("val")
ax.set_xlabel("i")

In [ ]:
expanding = np.array([(testdata[i]+i*np.random.randint(-1,1)*.2)
                     for i in np.arange(len(testdata))])
sns.distplot(expanding,kde=False,norm_hist=True)

In [ ]:
ax = sns.regplot(np.arange(len(expanding)),expanding,fit_reg=False)
ax.set_ylim(0,300)
ax.set_ylabel("val")
ax.set_xlabel("i")

# Lag Plot

* Plot point $y_i$ versus $y_{i-1}$
* If the lag plot is structureless
 * randomness assumption holds.

In [ ]:
lag = testdata.copy()
lag = np.array(lag[:-1])
current = testdata[1:]
ax = sns.regplot(current,lag,fit_reg=False)
ax.set_ylabel("y_i-1")
ax.set_xlabel("y_i")

In [ ]:
connected = np.array([testdata[i]+testdata[i-1] for i in np.arange(500)])
sns.distplot(connected,kde=False,norm_hist=True)

In [ ]:
lag = connected.copy()
lag = np.array(lag[:-1])
current = connected[1:]
ax = sns.regplot(current,lag,fit_reg=False)
ax.set_ylabel("y_i-1")
ax.set_xlabel("y_i")

## QQ Plot
* QQ Plots takes our n ordered data points
 * sorted from smallest to largest
* Asks:
 * What is the relationship between quantiles from our data and quantiles from a theoretical distribution that we're assuming the sample is drawn from

In [ ]:
qntls, xr = stats.probplot(testdata, fit=False)
sns.regplot(xr,qntls)

In [ ]:
def random_snorm(n, mean = 0, sd = 1, xi = 1.5):
    def random_snorm_aux(n, xi):
        weight = xi/(xi + 1/xi)
        z = np.random.uniform(-weight,1-weight,n)
        xi_ = xi**np.sign(z)
        random = -np.absolute(np.random.normal(0,1,n))/xi_ * np.sign(z)
        m1 = 2/np.sqrt(2 * np.pi)
        mu = m1 * (xi - 1/xi)
        sigma = np.sqrt((1 - m1**2) * (xi**2 + 1/xi**2) + 2 * m1**2 - 1)
        return (random - mu)/sigma

    return random_snorm_aux(n, xi) * sd + mean


In [ ]:
rightskewed = random_snorm(1000,xi=2)*100
sns.distplot(rightskewed,kde=False)

In [ ]:
qntls, xr = stats.probplot(rightskewed, fit=False)
sns.regplot(xr,qntls)

In [ ]:
leftskewed = random_snorm(1000,xi=-2)*100
sns.distplot(leftskewed,kde=False)

In [ ]:
qntls, xr = stats.probplot(leftskewed, fit=False)
sns.regplot(xr,qntls)

# <font color="red">BEGIN Q5:  Are we normal?</font>

## Now the serious plots... let's wrap them in a single function that we can call

In [ ]:
def multiplePlots(series):
    
    fig, axs = plt.subplots(2,2)
    plt.tight_layout(pad=0.4, w_pad=4, h_pad=1.0)

    # Histogram
    sns.distplot(series, ax=axs[0,0])
    
    # Lag plot code here
    
    # QQ plot code here

    # Run Sequence doe here



## Now run this on your "things"

In [ ]:
multiplePlots(things)

## <font color="red"><a href="https://www.theguardian.com/news/datablog/2010/jul/16/data-plural-singular">Do your data look</a> normally distributed?</font>
Explain why or why not.

### Normally Distributed Tomatoes
From the above plots we can see that the randome sample of tomato weights follows a mostly normal distribution with very minimal left skewing, most likely caused by the frequency of observations under 100 grams. We find that it is mostly normally distributed because there is no visible skewing on the QQ plot either.

# <font color="red">END Q5</font>

# Univariate Data -- Summary
* Simple but valuable
* We want to know how data is distributed
* How does it fit known models/distributions
* When does it not?
 * Visual and analytical tests

# <font color="red">BEGIN Q6: </font>
The sample.csv file, included with today's lab, contains 9 variables (v0 through v9) that contain
measures drawn from different distributions.
Your task is to use the investigative techniques we discussed in today's lab to determine
what type of distribution the sample is drawn from.

You should first load the CSV file into a DataFrame, then look at various aspects of **each** variable.
Your responses should consist of code cells, as well as markdown cells that state something like:
> Variable v99 appears to be drawn from a uniform distribution with mean X and standard deviation Y.  
> A histogram of the data appears to be...
> The QQ plot shows.... 


In [ ]:
#Enter your code hre

# END Q6

# ----------Vizualization----------

## Visualization for Data Scientists

We're going to ask a special virtual guest lecturer to provide some background on data visualization.  Together, we'll watch [a brief (8-video) by Dr. Chris Brooks](
https://www.coursera.org/learn/python-plotting/lecture/qrqqa/tools-for-thinking-about-design-alberto-cairo)
and pause it several times to answer the following questions:



## <font color="red">Q1a: As someone who is studying data science, who are you trying to reach through your visualizations?  </font>


_Explain your answer here._

## <font color="red">Q1b: What sense can you make of this image?</font>
![](resources/BrooksResearch.png)


_Explain your answer here._

## <font color="red">Q1c: How many different kinds of information can you see in the Minard graphic, and what are they?</font>

![](resources/Menard.png)

_Explain your answer here._


## Returning to Seaborn: 

https://seaborn.pydata.org/examples/index.html

Take a look at the different visualizations that are possible.

## <font color="red">Q2a: Provide the title, description, and URL of one of the visualizations that you find particularly interesting and explain why you find it interesting.  </font>

_Explain your answer here._

## <font color="red">Q2b: Given what we learned from Prof. Brooks, indicate 1-3 axes from Cairo's Visual Wheel where your chosen Seaborn visualization would likely score highly. Explain why.</font>

![](resources/CairoVisualWheel.png)

_Explain your answer here._

## Seaborn versus Matplotlib
* Matplotlib
     * Low-level, basis for many packages
     * Painful to construct certain graphs
     * Not Pandas friendly
     * Not interactive
* Seaborn
     * Pandas friendlier
     * Great for some stats plots


## Part 1: Iris dataset
![](resources/iris.png)

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
df = sns.load_dataset('iris')
df.head()

Remember our distplots:


In [ ]:
# Relationship between sepal length and width
sns.distplot(df.sepal_length)
sns.distplot(df.sepal_width)

## <font color="red"> Q3: Create similar plots for the other three numeric variables in the dataset. In a couple of sentences, describe each of the plots.  </font>

In [ ]:
#Enter your code here

_Explain your answer here._

We often want to see how variables vary with each other.  We'll get into the details 
in a few classes, but for now let's examine them visually.  In seaborn, we do this using 
the jointplot(). So, for example, if we wanted to look at the relationship between the
distributions of sepal_length and sepal_width, we could do something like:



In [ ]:
sns.jointplot(x='sepal_length',y='sepal_width',data=df)

## <font color="red"> Q4: It's a bit difficult to see where the interesting areas in the plot are, so it's worth trying a hexbin plot.  Go ahead and copy the above  code block and add ```kind="hex"``` to the jointplot parameters. In a couple of sentences, describe what stands out to you about the visualization. </font>

In [ ]:
#Enter your code here

_Explain your answer here._

Now, take a look at what happens when you set ```kind="kde"```

In [ ]:
sns.jointplot(x='sepal_length',y='sepal_width',data=df,kind="kde")

Finally, you may want to look at all the numeric variables in your
dataset. Use ```pairplot``` to do this:


In [ ]:
sns.pairplot(df.query("species == 'setosa'"))

We can get fancier by using a different column to set the color (or "hue"):

Try running the following code:

In [ ]:
sns.pairplot(df,hue="species")

Now let's introduce some correlations.  We're not going to spend time on learning about the 
theory behind correlation, as you've done that in the statistics prerequisite for this course.
Instead, we're going to jump right in and annotate a graph with a lot of statistical information:

In [ ]:
from scipy import stats

In [ ]:
# ignore the warning about deprecated annotation
g = sns.JointGrid(data=df,x='petal_length',y='sepal_length')
g = g.plot(sns.regplot, sns.distplot)
g = g.annotate(stats.pearsonr)

Think about what the different components mean.  We'll return to using this in the next section on Wine Quality.

## Part 2: Wine quality
![](resources/vinho.png)
https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009/home

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
wine = pd.read_csv('data/winequality-red.csv')
wine.head()

In [ ]:
wine['isgood'] = np.where(wine['quality'] > 5, 'good','bad')

In [ ]:
sns.distplot(wine['fixed acidity'])

In [ ]:
wine.skew()

## Q5: Create a pairplot for the wine dataset that plots 'good' and 'bad' wines in different hues. In a couple of sentences, describe interesting relationships shown by the visualization.  

In [ ]:
#Enter your code here

_Explain your answer here._

## T-test

A t-test is a simple statistical model that's commonly used to test whether the means of two different
distributions are the same.  scipy.stats gives us a handy interface for this:

In [ ]:
goodwines = wine.query('isgood == "good"')
badwines = wine.query('isgood == "bad"')

In [ ]:
stats.ttest_ind(wine[wine.isgood == 'good']['fixed acidity'],wine[wine.isgood == 'bad']['fixed acidity'])

## Q6: Using the JointGrid approach we used above look at the relationship between sulphates and chlorides.  What patterns do you see?

In [ ]:
#Enter your code hre

_Explain your answer here._

## Ordinary Least Squares (OLS) Regression

We can get a lot more detail about the regression model by using statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

statsmodels uses R-Style formula: y ~ x1 + x2 + x3 + ...

1. y represents the outcome/dependent variable
2. x1, x2, x3, etc represent explanatory/independent variables 

In [ ]:
model1 = smf.ols('chlorides ~ sulphates', data=wine).fit()
model1.summary()

# Commonly look at R-squared, F-statistic, coefficients

### Interesting things happen when we use OLS to do an ANOVA (look closely at the model):

In [ ]:
model2 = smf.ols('chlorides ~ isgood', data=wine).fit()
model2.summary()

### We might want to experiment with the original ```quality``` variable, either in a regression model:

In [ ]:
model3 = smf.ols('chlorides ~ quality', data=wine).fit()
model3.summary()

### or in an ANOVA (again, look closely at the model):

In [ ]:
model4 = smf.ols('chlorides ~ C(quality)', data=wine).fit() #Wrap it with C to convert to categorical
model4.summary()

## <font color="magenta">Q7: Use OLS to perform either a regression or an ANOVA on a variable (other than chlorides) and interpret your results.

In [ ]:
#Enter your code here

_Explain your answer here._

## Part 3:  Airplane Crashes and Fatalities
The next dataset we are going to look at is the full history of airplane crashes throughout the world, from 1908-2009.  It's taken from:

https://opendata.socrata.com/Government/Airplane-Crashes-and-Fatalities-Since-1908/q2te-8cvq

In [ ]:
import pandas as pd
import seaborn as sns

We've provided the CSV file for this lab so you can go ahead and load it in the usual way:

In [ ]:
crashes = pd.read_csv('data/Airplane_Crashes_and_Fatalities_Since_1908.csv')

As always, you should take a look at the data to get a sense of 
what it's like:

In [ ]:
crashes.head()

As we mentioned in an earlier class, pandas is really good at helping
us deal with dates.  The 'Date' column in the dataframe contains 
strings that look like dates.  We can use the ```pandas.to_datetime()``` function to convert the strings to an internal datetime object
(see https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html for more details):

In [ ]:
crashes['Date'] = pd.to_datetime(crashes['Date'])

And let's look at the dataframe again.  See any difference?

In [ ]:
crashes.head()

The pandas datetime object makes it easy to extract interesting 
parts of the date or time.  In our case, we're interested in extracting
the year, so we can do that with the following code:

In [ ]:
crashes['year'] = crashes['Date'].dt.year

And, as always, let's look at what we got:

In [ ]:
crashes.year.head()

As part of the final exercise in this class, let's create a 
visualization of the number of Fatalities per year:

In [ ]:
sns.barplot('year','Fatalities',data=crashes)

That doesn't look great, does it?  


## Q8: Create a barplot of the number of fatalities per decade and describe the results. 

Go ahead and create a new column called 'decade' 
that represents the decade for each year.  Remember that an integer divide (a.k.a. a floor divide) can be
done with the // operator.

What's the trend in airplane crash fatalities?

In [ ]:
#Enter your code here

_Explain your answer here._

## Part 4 (FYI): Functional Magnetic Resonance Imagining
![](resources/fmri.png)

In [ ]:
fmri = sns.load_dataset("fmri")

In [ ]:
fmri.head()

In [ ]:
fmri.describe()

In [ ]:
sns.relplot(x="timepoint", y="signal", kind="line", data=fmri);

In [ ]:
sns.relplot(x="timepoint", y="signal", kind="line", ci=None, data=fmri);

In [ ]:
sns.relplot(x="timepoint", y="signal", kind="line", ci="sd",data=fmri);

In [ ]:
sns.relplot(x="timepoint", y="signal", kind="line", estimator=None, data=fmri);

In [ ]:
sns.relplot(x = "timepoint", y = "signal", kind = "line", data = fmri, hue = "event");

In [ ]:
sns.relplot(x="timepoint", y="signal", kind="line", data=fmri, hue="region", style="event");